<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/2.%20Data%20Analysis/analysis_04_%EB%8F%85%EB%A6%BD%EC%98%81%ED%99%94%EB%B0%B0%EA%B8%89_%EB%B0%95%EC%9D%80%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [165]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

fatal: destination path 'mini-project-team1' already exists and is not an empty directory.


In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
pip install plotly==5.2.1

In [168]:
pip install pingouin

In [169]:
#기본
import pandas as pd 
import numpy as   np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pingouin as pg
import json
#plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as io
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

In [170]:
#영화 자료 가져오기 
movies = pd.read_csv("/content/mini-project-team1/1. Crawling/data/KOBIS_TOT.csv")
movies["개봉일"] = pd.to_datetime(movies["개봉일"])
movies.sample(1)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
390,391,2012,머신건 프리처,마크 포스터,(주)팝파트너스,2012-05-24,미국,141,197366500,26640,액션,15세이상관람가,일반영화


In [171]:
#배급사 자료 가져오기 (연도 별 상위 10개의 배급사)
distributor = pd.read_csv("/content/mini-project-team1/1. Crawling/data/2011_2020_topdistributor.csv")

#2015~2020 연도 별 독립영화 제작수와 개봉 수 비교 
movie_number = pd.read_csv("https://raw.githubusercontent.com/yujin-jo/mini-project-team1/main/1.%20Crawling/data/2015_2020_movienumbers.csv")
movie_number = movie_number.rename(columns={'구분':'개봉연도'})


---
# I. 가설 설정과 분석 배경

1. 가설  : 2014년 이후 대형 배급사가 독립∙예술 영화의 배급을 늘렸을 것이다.
    - 2011~2014년까지의 대형 배급사가 배급한 영화 수와 2015년 ~ 2020년 배급한 영화 수 사이에 유의미한 차이가 있을 것이다. 
    - 대형 배급사의 독립예술영화 배급 비중도 2014년 이전과 차이가 있을 것이다(증가했을 것이다)
    - 독립영화를 배급하는 배급사의 수와 비중도 2014년 이전과 차이가 있을 것이다 (감소했을 것이다).

    
2. 가설을 선택한 이유 : 
    - 2014년~2017년 사이에 대형 배급사 기준으로 독립영화 전용 스크린이 증가하기 시작 (CGV 아트하우스, 롯데시네마 이르떼클래식과 같은 브랜드가 생성) 
    - 2014년의 흥행 성적이 대형 배급사의 독립예술영화 배급에 영향을 미치지 않았을까? (이전과 이후를 비교하여 가설을 확인하고자 함) 
    - 2014년 이후 배급이 증가했다면, 일시적인 현상인지 아니면 향후 얼마 동안 대형 배급사의 의사결정에 영향을 주었을지 궁금했다. 
    - 대형 배급사에서의 독립예술영화 배급이 오히려 중소 독립영화 배급사의 파이를 뺏어간 것이 아닐까하는 궁금증 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 별 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별, 영화유형(독립예술영화, 일반영화)로 집계한 영화 수
  - 연도 별 매출 기준 TOP10 배급사 : 대형 배급사, 기타 배급사를 분리하여 파악하기 위함 

2. 분석 대상 
  - 비교 시기 : 1집단(2011년 ~ 2014년), 2집단(2015년~2020년) 
  - 비교 대상 
    - 대형 배급사가 배급한 영화 수 
    - 대형 배급사의 독립예술영화 배급 비중
    - 독립예술영화 중 배급사의 비중 비교 (대형배급사VS기타배급사)
    - 독립영화 배급사의 수
  
3. 분석 방법론 
  - 비교 분석(t검정) : 두 집단(비교 시기)의 집계(비교 대상)에 유의미한 차이가 있는지 비교 
  - 시계열 분석 : 2011년 ~ 2020년까지 전체적인 추이를 확인하고자함


4. 시각화 
  - 히스토그램 (영화 수)
  - 비율 바 차트 (대형 배급사의 독립 에술영화 배급 비중, 기간 별 / 독립예술영 화 중 대형 배급사의 배급수 비중, 기간 별) 
  - 파이차트 (대형 배급사의 독립 에술영화 배급 비중, 전체 / 독립예술영화 중 대형 배급사의 배급수 비중, 전체) 


5. 데이터 정의 
  - 대형 배급사 : 2011년 ~ 2020년 기간 동안 모두 매출액 기준 TOP10 내에 있었던 영화 배급사만을 기준으로 함(씨제이이엔엠, 롯데, NEW, 쇼박스, 워너브라더스)
  - 씨제이이엔앰의 경우 CGV계열사인 아트하우스 배급 영화도 포함
  - 2020년 영화 수가 급감했기 때문에 2020년은 제외할지 고민 중 (팀과 논의해서 배제 예정)



# III. 분석 과정  

## 0. 데이터 전처리 
### 1) 배급사 string 분리
  - 배급사 string을 , 기준으로 row로 분리하여 계산
  - 배급사가 분리된 데이터를 `new_movies`로 명명
  
### 2) 독립예술영화 데이터 분리 
  - 독립예술영화 데이터를 `movies_art`로 명명
  - 배급사가 분리된 예술영화 데이터를 `new_movies_art`로 명명


In [172]:
# 배급사명을 ,로 구분시키고 df화함
new_movies = movies.assign(배급사=movies.배급사.str.split(","))

# 분리된 배급사를 시리즈로 만들어준 후, 기존 테이블과 병합해줬다. 
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)

# 분리된 배급사들을 행으로 변환하기 위해 melt함수 적용해주고, 연도 별 영화 별로 쉽게 구분할 수 있도록 sorting해주었다. 
new_movies = new_movies.melt(id_vars = ["순번", "개봉연도", "영화명", "감독", "배급사", "개봉일", "국적", "전국스크린수", "전국매출액", "전국관객수", "장르", "등급", "영화구분"], var_name="영화N구분", value_name = "배급사명분리")
new_movies = new_movies.sort_values(by = ["순번", "영화N구분"], ascending = True)

# 영화 중 분리된 배급사 값이 Null값인 경우를 제외해 주었다
new_movies = new_movies[new_movies["배급사명분리"].notnull()]

# 배급사명을 원래 배급사명에 넣었다. 
new_movies['배급사'] = new_movies["배급사명분리"]

# 불필요한 컬럼을 제거했다.
new_movies.drop(['배급사명분리', '영화N구분'], axis=1, inplace=True)

# movies에서 독립예술영화만 분리 
movies_art = movies[movies['영화구분']=='독립/예술영화']

# new_moives에서 독립예술영화만 분리
new_movies_art = new_movies[new_movies['영화구분']=='독립/예술영화']

## 1. 데이터 테이블 overview
- 2011년 ~ 2020년까지의 총 3,375편의 영화 데이터가 들어감, 이 중 독립/예술 영화는 총 1,515편으로 44.9%를 차지한다.
- 독립예술 영화의 비중은 2014년까지 지속적으로 상승하다가 2015년에 소폭 하락하였고, 2016년부터 2019년까지 다시 지속적으로 상승하였음
- 2016년 이후 독립예술영화의 비중은 전체 영화의 50% 이상으로, 2011년 13%대에 비해 약 280% 성장
---
- 참고 테이블
  - `movies` : 전체 영화 데이터를 보여주는 테이블 
  - `movies_art` : 독립/예술 영화로 분류된 영화 정보를 보여주는 테이블 
  - `new_movies` : 배급사가 분리된 전체 영화 데이터를 보여주는 테이블
  - `new_movies_art` : 배급사가 분리된 독립/에술영화 데이터를 보여주는 테이블

In [173]:
# 영화 기본 정보 확인 
movies.info(),  movies_art.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3375 non-null   int64         
 1   개봉연도    3375 non-null   int64         
 2   영화명     3375 non-null   object        
 3   감독      3375 non-null   object        
 4   배급사     3374 non-null   object        
 5   개봉일     3375 non-null   datetime64[ns]
 6   국적      3375 non-null   object        
 7   전국스크린수  3375 non-null   int64         
 8   전국매출액   3375 non-null   int64         
 9   전국관객수   3375 non-null   int64         
 10  장르      3375 non-null   object        
 11  등급      3375 non-null   object        
 12  영화구분    3375 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 342.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 29 to 3373
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  -----

(None, None)

In [174]:
#연도 별 영화 수와 독립영화 비중을 확인 (movie_transition테이블을 만들어서 확인)
#독립예술 영화의 비중은 2014년까지 지속적으로 상승하다가 2015년에 소폭 하락하였고, 2016년부터 2019년까지 다시 지속적으로 상승하였음
#2016년 이후 독립예술영화의 비중은 전체 영화의 50% 이상으로, 2011년 13%대에 비해 약 280% 성장
movie_transition1 = movies.groupby(['개봉연도'])['영화명'].size().reset_index(name='총영화수')
movie_transition2 = movies.groupby(['개봉연도', '영화구분'])['영화명'].size().reset_index(name="유형별영화수")
movie_transition = pd.merge(movie_transition2, movie_transition1, how = 'left', on = '개봉연도')
movie_transition['비중'] = movie_transition['유형별영화수'] / movie_transition['총영화수']*100
fig = px.bar(movie_transition,
             x = "개봉연도", 
             y = "유형별영화수", 
             text = "유형별영화수",
             title="연도 별 영화 구분에 따른 영화 수 추이", 
             color = "영화구분", hover_name = "영화구분", 
             hover_data=["비중"], log_x = True)
fig.update_traces(texttemplate='%{text}개', textposition='inside')
fig.show()

In [175]:
pd.read_csv("/content/mini-project-team1/1. Crawling/data/2011_2020_topdistributor.csv")

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,씨제이,씨제이,씨제이,씨제이,씨제이,씨제이,씨제이,롯데,월트디즈니,씨제이
1,롯데,쇼박스,NEW,롯데,쇼박스,쇼박스,쇼박스,씨제이,씨제이,롯데
2,NEW,롯데,롯데,소니,월트디즈니,월트디즈니,롯데,NEW,롯데,NEW
3,소니,NEW,쇼박스,워너브라더스,이십세기폭스,워너브라더스,NEW,쇼박스,NEW,쇼박스
4,쇼박스,소니,소니,이십세기폭스,유니버설픽쳐스,이십세기폭스,메가박스,메가박스,워너브라더스,워너브라더스
5,이십세기폭스,워너브라더스,워너브라더스,쇼박스,NEW,NEW,아트하우스,워너브라더스,쇼박스,UPI
6,워너브라더스,유니버설픽쳐스,유니버설픽쳐스,NEW,롯데,롯데,오퍼스,키다리,소니,월트디즈니
7,유니버설,이십세기폭스,이십세기폭스,유니버설픽쳐스,워너브라더스,유니버설픽쳐스,리틀빅,리틀빅,유니버설픽쳐스,에이스케이커
8,시너지,필라멘트픽쳐스,아이러브시네마,아트하우스,아트하우스,메가박스,워너브라더스,아트하우스,에이스케이커,메가박스
9,화앤담,에스비에스,씨너스,판씨네마,메가박스,와우픽쳐스,키다리,옛나인,이십세기폭스,소니


## 2. 데이터 분석 

### 1) 2011 ~ 2020년 대형 배급사와 기타 배급사의 배급 영화수 및 매출액 비교 (전체 기간, 시간 별 추이)
- 2011년 ~ 2020년 간 제작된 독립예술영화 편수에서 대형 배급사는 전체의 6.09%를 차지하지만, 매출액에서 차지하는 비중은 17.7%로, 대형 배급사의 영화 1편 당 수익성이 더 나은 것으로 확인할 수 있었다.
- 2011년 ~ 2015년까지 대형 배급사의 독립예술영화 편수는 증감이 반복되는 추세이며, 2016년 이후 꾸준한 감소세를 보임. 2020년에는 2편만 배급했음 
- 2019년에 CGV 아트하우스가 독립예술영화 배급을 중단하면서 대형 배급사에서 독립예술영화 배급수가 급감한 것으로 판단됨 

In [176]:
# 독립예술영화 중 대형 배급사가 배급한 영화를 '배급사구분'에 표시했다. 
# big5 : 씨제이', '롯데', 'NEW', '쇼박스', '워너브라더스, '아트하우스'
new_movies_art['배급사구분'] = np.where(new_movies_art['배급사'].str.contains('씨제이|롯데|NEW|쇼박스|워너|아트하우스'), '대형배급사', '기타배급사')
new_movies_art.head(5)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분,배급사구분
29,30,2011,그대를 사랑합니다,추창민,(주)넥스트엔터테인먼트월드(NEW),2011-02-17,한국,242,12005917000,1640351,드라마,15세이상관람가,독립/예술영화,대형배급사
71,72,2011,세 얼간이,라즈쿠마르 히라니,필라멘트픽쳐스,2011-08-18,인도,232,3352570500,458988,코미디,12세이상관람가,독립/예술영화,기타배급사
79,80,2011,내 이름은 칸,카란 조하르,필라멘트픽쳐스,2011-03-24,인도,84,2820922500,381243,드라마,12세이상관람가,독립/예술영화,기타배급사
116,117,2011,50/50,조나단 레빈,롯데쇼핑㈜롯데엔터테인먼트,2011-11-24,미국,159,1019768500,135985,코미디,15세이상관람가,독립/예술영화,대형배급사
118,119,2011,제인 에어,캐리 후쿠나가,(주)시너지하우스 (시너지),2011-04-20,영국,208,981132000,135797,드라마,12세이상관람가,독립/예술영화,기타배급사


In [177]:
#2011년 ~ 2020년 간 제작된 독립예술영화의 배급 비중 (대형 배급사는 전체의 6.09% 차지)
new_movies_art_counts_tot = new_movies_art.groupby('배급사구분').size().reset_index(name='배급영화수')
# new_movies_art_counts['총영화수'] = new_movies_art_counts.groupby('개봉연도')['배급영화수'].transform('sum')
# new_movies_art_counts['비중'] = new_movies_art_counts['배급영화수']/new_movies_art_counts['총영화수']*100
new_movies_art_counts_tot

fig = px.pie(new_movies_art_counts_tot, values='배급영화수', names='배급사구분',
             color_discrete_sequence=px.colors.sequential.Redor,
             hole=.3,
             title='2011년~2020년간 제작된 독립예술영화의 배급 비중')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [178]:
new_movies_art_sum = new_movies_art.groupby(['개봉연도','배급사구분'])['전국매출액'].sum().reset_index(name='매출액총계')

In [ ]:
# 대형 배급사는 배급한 영화 수에 비해서 영화 별 매출액이 기타 배급사보다 높은 것을 확인할 수 있다. 
new_movies_art_sum_tot = new_movies_art.groupby('배급사구분')['전국매출액'].sum().reset_index(name='매출액')
fig = px.pie(new_movies_art_sum_tot, values='매출액', names='배급사구분',
             color_discrete_sequence=px.colors.sequential.Redor,
             hole=.3,
             title='2011년~2020년간 제작된 독립예술영화의 배급사 구분 별 매출액 비중')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [179]:
#배급사 구분 별로 매년 배급 영화수와 총 영화수 비교
new_movies_art_counts = new_movies_art.groupby(['개봉연도','배급사구분']).size().reset_index(name='배급영화수')
new_movies_art_counts['총영화수'] = new_movies_art_counts.groupby('개봉연도')['배급영화수'].transform('sum')
new_movies_art_counts['배급비중'] = new_movies_art_counts['배급영화수']/new_movies_art_counts['총영화수']*100

#배급사 구분 별로 매년 배급되는 영화의 영화 매출액 비교 
new_movies_art_sum = new_movies_art.groupby(['개봉연도','배급사구분'])['전국매출액'].sum().reset_index(name='배급사매출액')
new_movies_art_sum['총매출액'] = new_movies_art_sum.groupby('개봉연도')['배급사매출액'].transform('sum')
new_movies_art_sum['매출액비중'] = new_movies_art_sum['배급사매출액']/new_movies_art_sum['총매출액']*100


new_movies_art_agg = pd.merge(new_movies_art_counts, new_movies_art_sum, how="left", on=["개봉연도", "배급사구분"])
new_movies_art_agg[['개봉연도', '배급사구분', '배급영화수', '배급비중', '배급사매출액','매출액비중']]
# new_movies_art_counts

,개봉연도,배급사구분,배급영화수,배급비중,배급사매출액,매출액비중
0,2011,기타배급사,29,90.625000,13400617900,49.001787
1,2011,대형배급사,3,9.375000,13946584500,50.998213
2,2012,기타배급사,45,90.000000,26211994507,83.280991
3,2012,대형배급사,5,10.000000,5262167971,16.719009
4,2013,기타배급사,76,87.356322,50213777342,95.294254
5,2013,대형배급사,11,12.643678,2479617141,4.705746
6,2014,기타배급사,161,92.000000,120038064846,76.802878
7,2014,대형배급사,14,8.000000,36255642495,23.197122
8,2015,기타배급사,178,93.684211,57501399938,76.127786
9,2015,대형배급사,12,6.315789,18031336460,23.872214


In [180]:
#배급사 별로 매년 배급 영화 수(비중)이 어떻게 달라지는지 살펴보면 연도 별로 비중 편차가 심한 편이며, 2020년에는 독립 영화 중 1% 이하를 대형 배급사에서 차지했다. 
fig = px.bar(new_movies_art_agg,
             x = "개봉연도", 
             y = "배급영화수", 
            #  text = "배급영화수",
             title="연도 별 배급사에 따른 독립예술 영화 수 추이", 
             color = "배급사구분", hover_name = "배급사구분", 
             hover_data=["배급비중"], log_x = True)
# fig.update_traces(texttemplate='%{text}개', textposition='inside')
fig.show()

In [181]:
#배급사 별로 매년 배급 영화의 매출액을 비교하였으나, 매출액 비중에서의 추이 규칙성을 찾을 수 없었음 
fig = px.bar(new_movies_art_agg,
             x = "개봉연도", 
             y = "배급사매출액", 
            #  text = "배급사매출액",
             title="연도 별 배급사에 따른 독립예술 영화 매출 추이", 
             color = "배급사구분", hover_name = "배급사구분", 
             hover_data=["매출액비중"], log_x = True)
# fig.update_traces(texttemplate='%{text}원', textposition='inside')
fig.show()

In [182]:
# 대형 배급사의 파이에서 CJ 계열이 얼마나 많은 영향력을 행사했는지 확인해봤다. 
# 영화 편수를 봤을 때 CJ계열이 차지하는 비중은 60%로 과반수 이상을 차지하는 것을 확인할 수 있다. 특히 이 중 CGV아트하우스는 전채 편수의 49.5%를 담당했다. 
new_movies_art_big5 = new_movies_art[new_movies_art['배급사구분']=='대형배급사']
new_movies_art_big5_count_tot = new_movies_art_big5.groupby('배급사').size().reset_index(name='배급영화수')


fig = px.pie(new_movies_art_big5_count_tot, values='배급영화수', names='배급사',
             color_discrete_sequence=px.colors.sequential.Magenta_r,
             hole=.3,
             title='2011년~2020년간 제작된 독립예술영화의 배급사 구분 별 배급편수 비중')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [183]:
# 매출액 비중을 보더라도 CGV+CGV 아트하우스가 2011년 ~ 2020년 매출액의 49.07%를 차지하고 있음
new_movies_art_big5 = new_movies_art[new_movies_art['배급사구분']=='대형배급사']

fig = px.pie(new_movies_art_big5, values='전국매출액', names='배급사',
             color_discrete_sequence=px.colors.sequential.Magenta_r,
             hole=.3,
             title='2011년~2020년간 제작된 독립예술영화의 배급사 구분 별 매출액 비중')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [184]:
#대형 배급사 별로 매년 배급 영화수와 총 영화수 비교
new_movies_art_big5_counts = new_movies_art_big5.groupby(['개봉연도','배급사']).size().reset_index(name='배급영화수')
new_movies_art_big5_counts['총영화수'] = new_movies_art_big5_counts.groupby('개봉연도')['배급영화수'].transform('sum')
new_movies_art_big5_counts['배급비중'] = new_movies_art_big5_counts['배급영화수']/new_movies_art_big5_counts['총영화수']*100
new_movies_art_big5_counts


#배급사 구분 별로 매년 배급되는 영화의 영화 매출액 비교 
new_movies_art_big5_sum = new_movies_art_big5.groupby(['개봉연도','배급사'])['전국매출액'].sum().reset_index(name='배급사매출액')
new_movies_art_big5_sum['총매출액'] = new_movies_art_big5_sum.groupby('개봉연도')['배급사매출액'].transform('sum')
new_movies_art_big5_sum['매출액비중'] = new_movies_art_big5_sum['배급사매출액']/new_movies_art_big5_sum['총매출액']*100


new_movies_art_big5_agg = pd.merge(new_movies_art_big5_counts, new_movies_art_big5_sum, how="left", on=["개봉연도", "배급사"])
new_movies_art_big5_agg = new_movies_art_big5_agg[['개봉연도', '배급사', '배급영화수', '배급비중', '배급사매출액','매출액비중']].sort_values(by = ["개봉연도", "배급영화수"], ascending=[True, False])


In [185]:
#배급사 별로 매년 배급 영화 수(비중) 추이를 보면 CGV 아트하우스에서 30~70%의 영화 배급을 담당했음을 알 수 있음 
#아트하우스가 들어온 2013년에는 영화 배급 수가 5편에서 11편으로 120% 상승하는데, 이때 CGV가 4편이 증가하면서 영화 배급수 상승의 60%를 차지함
#아트하우스가 철수한 2020년에는 영화 배급 수 차이가 가장 큰 것을 확인할 수 있음 (대형배급사의 배급 영화 수는 14편에서 2편으로 80% 이상 감소했는데, CGV 아트하우스가 7편이 감소하면서 감소량의 58.3%를 설명함  )
fig = px.bar(new_movies_art_big5_agg,
             x = "개봉연도", 
             y = "배급영화수", 
            #  text = "배급영화수",
             title="연도 별 배급사에 따른 독립예술 영화 수 추이", 
             color = "배급사", hover_name = "배급사", color_discrete_sequence=px.colors.sequential.Oranges,
             hover_data=["배급비중"], log_x = True)
# fig.update_traces(texttemplate='%{text}개', textposition='inside')
fig.show()

In [186]:
#배급사 별로 매년 배급 영화 매출 추이를 보면 CGV 아트하우스는 1편 당 매출액이 40억 이상인 2014년을 제외('님아 그 강을 건너지 마오')하고는 영화 1편 당 매출이 7억 대로 감소하고 있었고, 2017년에는 매출이 3.5억, 2018년에는 매출이 2.7억원으로 감소 
#cgv에 따르면 투자, 배급한 영화의 상당수 작품이 손익 분기점을 넘기지 못함 -> 지속되는 적자를 해결하지 못하고 2019년 사업 철회
fig = px.bar(new_movies_art_big5_agg,
             x = "개봉연도", 
             y = "배급사매출액", 
            #  text = "배급영화수",
             title="연도 별 배급사에 따른 독립예술 영화 매출액 추이", 
             color = "배급사", hover_name = "배급사", color_discrete_sequence=px.colors.sequential.Oranges,
             hover_data=["매출액비중"], log_x = True)
# fig.update_traces(texttemplate='%{text}개', textposition='inside')
fig.show()

In [187]:
#어떤 영화를 배급했는 지에 따라 그해 영화의 수익성이 결정남 -> 소수의 흥행 영화에 의존하여 나머지 배급의 적자를 메우는 꼴 
#NEW 2015 위플래시, CGV 아트하우스 2014 님아, 그 강을 건너지 마오, 2015 소셜포비아, 2016 캐롤, 카페 소사이어티, 2019 메기, 롯데 2019 항거
new_movies_art_big5.groupby(['배급사','개봉연도'])['영화명'].apply(list).reset_index(name="대표 배급영화")


,배급사,개봉연도,대표 배급영화
0,(주)넥스트엔터테인먼트월드(NEW),2011,"[그대를 사랑합니다, 삼국지: 명장 관우]"
1,(주)넥스트엔터테인먼트월드(NEW),2012,[피에타]
2,(주)넥스트엔터테인먼트월드(NEW),2013,"[블랙 가스펠, 뫼비우스, 사이비]"
3,(주)넥스트엔터테인먼트월드(NEW),2014,[원 챈스]
4,(주)넥스트엔터테인먼트월드(NEW),2015,"[지금은맞고그때는틀리다, 명량: 회오리 바다를 향하여]"
5,(주)넥스트엔터테인먼트월드(NEW),2016,"[서울역, 그물, 잔예 - 살아서는 안되는 방]"
6,(주)넥스트엔터테인먼트월드(NEW),2017,[너의 췌장을 먹고 싶어]
7,(주)넥스트엔터테인먼트월드(NEW),2018,"[당갈, 너의 췌장을 먹고 싶어, 펭귄 하이웨이, 쏘아올린 불꽃, 밑에서 볼까? 옆..."
8,(주)넥스트엔터테인먼트월드(NEW),2019,"[안녕, 티라노: 영원히, 함께, 콜레트, 시크릿 슈퍼스타]"
9,(주)쇼박스,2014,[그레이스 오브 모나코]


In [188]:
# # 한국 독립영화의 제작 편수는 증가하고 있는 데 반해 개봉 편수가 줄어들면서, 독립예술영화의 갈길이 더욱 험난해지고 있음 
# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x = movie_number['개봉연도'],
#         y = movie_number['제작편수 대비 개봉편수 비중'],
#         name="제작편수 대비 개봉편수 비중",
#         text="제작편수 대비 개봉편수 비중",
#         mode='lines+markers',
#     ))


# fig.add_trace(
#     go.Bar(
#         x = movie_number['개봉연도'],
#         y = movie_number['개봉편수'],
#         name = "개봉편수",
#         opacity = 0.3

#     ))

# fig.show()

### 2) 2014년 이전과 2015년 이후 배급 영화수에 차이가 있었는지
- 비교시기 1(2011년 ~ 2014년), 비교시기 2(2015년 ~ 2020년)
- 비교시기 1과 비교시기 2의 평균 영화 배급수의 차이를 비교한 결과 


In [195]:
# 독립예술영화 중 대형 배급사가 배급한 영화를 '배급사구분'에 표시한 것이 new_movies_art 테이블
# new_movies_art['개봉연도구분'] = np.where(new_movies_art['개봉연도'].str.contains('2011|2012|2013|2014'), '대형배급사', '기타배급사')
new_movies_art_big5_years = new_movies_art_big5[new_movies_art_big5['배급사구분']=='대형배급사']
new_movies_art_big5_years['개봉연도구분'] = np.where(new_movies_art_big5_years['개봉연도']<=2014, '2014년이전', '2015년이후')
new_movies_art_big5_years.head(2)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분,배급사구분,개봉연도구분
29,30,2011,그대를 사랑합니다,추창민,(주)넥스트엔터테인먼트월드(NEW),2011-02-17,한국,242,12005917000,1640351,드라마,15세이상관람가,독립/예술영화,대형배급사,2014년이전
116,117,2011,50/50,조나단 레빈,롯데쇼핑㈜롯데엔터테인먼트,2011-11-24,미국,159,1019768500,135985,코미디,15세이상관람가,독립/예술영화,대형배급사,2014년이전


In [200]:
new_movies_art_big5_years
#배급사 구분 별로 매년 배급 영화수와 총 영화수 비교
new_movies_art_big5_years_counts = new_movies_art_big5_years.groupby('개봉연도구분').size().reset_index(name='배급영화수')
# new_movies_art_big5_years_counts['전국스크린수'] = new_movies_art_big5_years.groupby('개봉연도구분').sum().reset_index(name='배급영화수')
new_movies_art_big5_years_counts['스크린수'] = new_movies_art_big5_years.groupby('개봉연도구분')['전국스크린수'].sum()
new_movies_art_big5_years_counts

,개봉연도구분,배급영화수,스크린수
0,2014년이전,33,NaN
1,2015년이후,72,NaN


In [ ]:
#이상한 히스토그램이 만들어짐
fig = px.histogram(new_movies_art_big5_years, x="전국매출액", color="개봉연도구분", barmode='group')
fig.show()

In [ ]:
#전국 
import pingouin as pg
aov = pg.anova(data=new_movies_art, dv='전국매출액', between='개봉연도구분', detailed=True)
aov

#2014년 이전과 2015년 이후에 대형 배급사의 매출액 사이에 유의미한 차이가 있다. 



,Source,SS,DF,MS,F,p-unc,np2
0,개봉연도구분,6.858613e+19,1,6.858613e+19,30.077583,4.767047e-08,0.017167
1,Within,3.926689e+21,1722,2.280307e+18,NaN,NaN,NaN


In [ ]:
4.767047e-08<0.05

True

In [ ]:
ttest = pg.ttest(new_movies_art_big5_years['전국매출액'], 0, confidence=0.95)

TypeError: ignored